<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_30WMA_RSI14_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2023-10-20 13:51:06


In [2]:
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
import pandas as pd
import numpy as np
#from google.colab import drive
#drive.mount('/content/drive')

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/'

mypf = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

#mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC','JIOFIN','SCHAEFFLER ']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

all_prospects = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

all_prospects.sort()
#stock_ids

In [4]:
# import necessary libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [5]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [6]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 10
      #mid_window = 50
      long_window = 30
      moving_avg = 'WMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='5y', interval='1wk')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['30_WMA'] = round(stock_df['Close'].ewm(span=long_window, adjust=False).mean(),0)
      stock_df['Dev%_30'] = round((stock_df['Close'] - stock_df['30_WMA'])*100/stock_df['30_WMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),2)
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
      stock_df['Min'] = round(min(stock_df['Close']),2)
      stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in all_prospects:
      #print(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      df_prec_dev = df_prec_dev.append(tmp)
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()


def get_stock_buy_sell(stock_symbol):
    #date = (datetime.date.today() + datetime.timedelta(days=0))
    df_prec_dev = stock_prec_dev(stock_symbol)
    df_prec_dev['Signal'] = 0.0
    df_prec_dev['Signal'] = np.where(((df_prec_dev['Close'] > df_prec_dev['30_WMA']) & (df_prec_dev['RSI_14'] > 60) ), 1.0, df_prec_dev['Signal'])

    df_prec_dev['Signal'] = np.where(((df_prec_dev['Close'] < df_prec_dev['30_WMA']) & (df_prec_dev['RSI_14'] < 60) ), -1.0, df_prec_dev['Signal'])

    df_prec_dev['Position'] = df_prec_dev['Signal'].diff()

    df_pos = df_prec_dev[(df_prec_dev['Position'] >= 1) | (df_prec_dev['Position'] <= -1)]
    df_pos = df_pos[df_pos['Signal'] != 0]
    df_pos['Prev_Signal'] = df_pos['Signal'].shift(1)
    df_pos = df_pos[df_pos['Signal'] != df_pos['Prev_Signal']]
    df_pos.drop('Prev_Signal', axis=1, inplace=True)

    return df_pos.tail(1)

def get_weekly_buy_sell():
    df_buy_sell = pd.DataFrame()
    for stock_id in all_prospects:
        df_tmp = get_stock_buy_sell(stock_id+'.NS')
        df_tmp['Criteria'] = mypf[(mypf['Symbol']) == stock_id.upper()]['Criteria'].values[0]
        df_buy_sell = df_buy_sell.append(df_tmp)


    df_buy_sell['Position'] = df_buy_sell['Position'].apply(lambda x: 'Buy' if x >= 1 else 'Sell')
    return df_buy_sell.sort_index()


In [7]:
df_weekly_buy_sell = get_weekly_buy_sell()
df_weekly_buy_sell

,Close,30_WMA,Dev%_30,Stock,Max,Min,Gained%,RSI_14,Signal,Position,Criteria
Date,,,,,,,,,,,
2023-01-30 00:00:00+05:30,76.0,72.0,5.64,IOC.NS,96.33,66.37,14.60,62.0,1.0,Buy,NaN
2023-01-30 00:00:00+05:30,4831.0,5627.0,-14.14,FINEORG.NS,7060.06,4121.44,17.22,21.0,-1.0,Sell,V40N
2023-01-30 00:00:00+05:30,286.0,235.0,21.67,INDIANB.NS,426.05,237.23,20.52,65.0,1.0,Buy,NaN
2023-01-30 00:00:00+05:30,372.0,319.0,16.75,ITC.NS,490.45,319.66,16.51,73.0,1.0,Buy,V40
2023-01-30 00:00:00+05:30,2142.0,1975.0,8.47,LT.NS,3093.70,1952.97,9.70,62.0,1.0,Buy,V200
...,...,...,...,...,...,...,...,...,...,...,...
2023-10-16 00:00:00+05:30,819.0,833.0,-1.66,VOLTAS.NS,1303.68,737.11,11.14,48.0,-1.0,Sell,NaN
2023-10-16 00:00:00+05:30,925.0,936.0,-1.14,FINCABLES.NS,1126.60,506.19,82.80,47.0,-1.0,Sell,V40N
2023-10-16 00:00:00+05:30,563.0,574.0,-1.86,SBIN.NS,616.05,495.72,13.64,45.0,-1.0,Sell,VR


In [8]:
df_weekly_buy_sell[df_weekly_buy_sell['Position'] == 'Sell'].tail(20)

,Close,30_WMA,Dev%_30,Stock,Max,Min,Gained%,RSI_14,Signal,Position,Criteria
Date,,,,,,,,,,,
2023-09-18 00:00:00+05:30,3416.0,3512.0,-2.74,PIIND.NS,3910.64,2925.79,16.74,45.0,-1.0,Sell,NaN
2023-09-25 00:00:00+05:30,1608.0,1627.0,-1.14,BATAINDIA.NS,2085.53,1389.69,15.74,48.0,-1.0,Sell,V40
2023-09-25 00:00:00+05:30,3161.0,3174.0,-0.41,ASIANPAINT.NS,3527.20,2704.64,16.88,49.0,-1.0,Sell,V40
2023-09-25 00:00:00+05:30,4165.0,4223.0,-1.37,NAUKRI.NS,6945.78,3443.25,20.97,48.0,-1.0,Sell,NaN
2023-09-25 00:00:00+05:30,552.0,555.0,-0.61,DABUR.NS,642.21,502.64,9.74,48.0,-1.0,Sell,V40
2023-10-02 00:00:00+05:30,537.0,542.0,-0.84,MARICO.NS,586.65,473.25,13.57,48.0,-1.0,Sell,V40
2023-10-02 00:00:00+05:30,3533.0,3580.0,-1.31,SIEMENS.NS,3945.60,2747.94,28.57,47.0,-1.0,Sell,NaN
2023-10-09 00:00:00+05:30,523.0,529.0,-1.21,ICICIPRULI.NS,702.11,393.60,32.77,48.0,-1.0,Sell,V40
2023-10-09 00:00:00+05:30,22389.0,22405.0,-0.07,ABBOTINDIA.NS,24059.95,18678.81,19.86,49.0,-1.0,Sell,V40


In [9]:
df_weekly_buy_sell[df_weekly_buy_sell['Position'] == 'Buy'].tail(20)

,Close,30_WMA,Dev%_30,Stock,Max,Min,Gained%,RSI_14,Signal,Position,Criteria
Date,,,,,,,,,,,
2023-08-14 00:00:00+05:30,1257.0,1108.0,13.45,JCHAC.NS,3159.85,951.65,32.09,66.0,1.0,Buy,V40N
2023-08-14 00:00:00+05:30,569.0,504.0,12.83,WSTCSTPAPR.NS,759.50,459.99,23.62,62.0,1.0,Buy,VR
2023-08-14 00:00:00+05:30,55.0,48.0,14.17,ASIANTILES.NS,300.21,35.05,56.35,63.0,1.0,Buy,VR
2023-08-21 00:00:00+05:30,141.0,129.0,9.07,FEDERALBNK.NS,149.40,121.24,16.05,63.0,1.0,Buy,NaN
2023-08-21 00:00:00+05:30,117.0,110.0,6.04,NMDC.NS,165.19,95.46,22.20,65.0,1.0,Buy,VR
2023-08-21 00:00:00+05:30,68.0,62.0,8.87,NETWORK18.NS,111.80,52.25,29.19,61.0,1.0,Buy,NaN
2023-08-28 00:00:00+05:30,81.0,67.0,21.12,IBREALEST.NS,189.60,48.90,65.95,63.0,1.0,Buy,NaN
2023-08-28 00:00:00+05:30,182.0,166.0,9.46,INDUSTOWER.NS,300.31,137.00,32.63,61.0,1.0,Buy,NaN
2023-08-28 00:00:00+05:30,688.0,640.0,7.55,IRCTC.NS,1078.37,568.14,21.16,62.0,1.0,Buy,AR


In [10]:
#tmp_df = df_daily_buy_sell.reset_index()
#tmp_df.to_csv('/content/drive/My Drive/data/stocks/myProspects-Buy-Sell-Signals.csv', index=False)

In [11]:
#stock_symbol = 'INFY.NS'
#plot_stock_buy_sell(stock_symbol)